# Create the features DF
* using by_postal_code dataset

# Import

In [1]:
import pandas as pd
import json
import numpy as np
from collections import Counter
import pickle
import math 
from sklearn.model_selection import train_test_split
import torch

In [2]:
RANDOM_STATE = 24

In [3]:
# 導入資料
train_df = pd.read_pickle('../data/Train_by_postoal_code_without_review_pointwise_v3_3.pkl').reset_index(drop=True)
test_df = pd.read_pickle('../data/Test_by_postoal_code_without_review_pointwise_v3_3.pkl').reset_index(drop=True)
all_df = pd.read_pickle('../Data/restaurant_only_filtered_with_embedding.pkl')

In [4]:
print(train_df.shape, test_df.shape, all_df.shape)

(11324, 10) (7098, 10) (387723, 22)


## Get Review Features

In [5]:
# Example list of sets
# sets_list = list(train_df[:5].LDA_res)
# combine_sets(sets_list)

def combine_sets(sets_list):
    # Create an empty dictionary to store the combined values
    combined_values = {}

    # Iterate over each set in the list
    for s in sets_list:
        # Iterate over each key-value pair in the set
        for key, value in s:
            # Check if the key already exists in the dictionary
            if key in combined_values:
                # Add the value to the existing key
                combined_values[key].append(value)
            else:
                # Add a new key-value pair to the dictionary
                combined_values[key] = []
                combined_values[key].append(value)
    # Create a set from the combined key-value pairs
    # combined_set = set(combined_values.items())

    for key , val in combined_values.items():
        combined_values[key] = np.mean(val)
        
    return combined_values


In [6]:
def get_LDA_aspects(df , all_df):

    df['LDA_res'] = ''
    df['LDA_loc'] = ''

    for idx , row in df.iterrows():
        if idx % 1000 ==0 :
            print(f'Now progress... {idx}')
        res = all_df[all_df.name == df.name[idx]]
        res = res[res.postal_code != row.postal_code]
        loc = all_df[all_df.postal_code == row.postal_code]
        loc = loc[loc.name != df.name[idx]]
        
        res_sets_list = list(res.LDA_aspects)
        loc_sets_list = list(loc.LDA_aspects)
        
        df['LDA_res'][idx] = combine_sets(res_sets_list)
        df['LDA_loc'][idx] = combine_sets(loc_sets_list)

    return df


In [7]:
train_df = get_LDA_aspects(train_df , all_df)
test_df = get_LDA_aspects(test_df , all_df)

Now progress... 0


/mnt/data/emma/bilab/Steph_C/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/mnt/data/emma/bilab/Steph_C/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Now progress... 1000
Now progress... 2000
Now progress... 3000
Now progress... 4000
Now progress... 5000
Now progress... 6000
Now progress... 7000
Now progress... 8000
Now progress... 9000
Now progress... 10000
Now progress... 11000
Now progress... 0
Now progress... 1000
Now progress... 2000
Now progress... 3000
Now progress... 4000
Now progress... 5000
Now progress... 6000
Now progress... 7000


In [8]:

def combine_bert_embedding(df , all_df):
    
    df['res_emb'] = ''
    df['loc_emb'] = ''
    
    for idx , row in df.iterrows():
        
        if idx % 1000 ==0 :
            print(f'Now progress... {idx}')
        
        res = all_df[all_df.name == df.name[idx]]
        res = res[res.postal_code != row.postal_code]
        loc = all_df[all_df.postal_code == row.postal_code]
        loc = loc[loc.name != df.name[idx]]
        
        res_embedding = list(res.BERT_embedding)
        loc_embedding = list(loc.BERT_embedding)
        print(len(torch.mean(torch.stack(res_embedding), dim=0)))
        df['res_emb'][idx] = torch.mean(torch.stack(res_embedding), dim=0)
        df['loc_emb'][idx] = torch.mean(torch.stack(loc_embedding), dim=0)
        
        
    return df

In [9]:
train_df = combine_bert_embedding(train_df , all_df)
test_df = combine_bert_embedding(test_df , all_df)

Now progress... 0
768


/mnt/data/emma/bilab/Steph_C/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/mnt/data/emma/bilab/Steph_C/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768


768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768


768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768


...

768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768


768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
Now progress... 1000
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768

768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
Now progress... 3000
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768

768
768
768
768
768
768
768
768
768
768
768
768
Now progress... 5000
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768

768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768


In [10]:
def get_affinity_complementary(df):
    df['affinity'] = ''
    df['complementary'] = ''
    
    for idx , row in df.iterrows():
        affinity , complementary = 0,0
        if idx % 1000 ==0 :
            print(f'Now progress... {idx}')
        for res_key , res_val in df.LDA_res[idx].items():
            if res_key in df.LDA_loc[0]:
                affinity += res_val*train_df.LDA_loc[0][res_key]
                complementary += res_val*(1-train_df.LDA_loc[0][res_key])
        df['affinity'][idx] = affinity
        df['complementary'][idx] = complementary
        
    return df

In [11]:
train_df = get_affinity_complementary(train_df)
test_df = get_affinity_complementary(test_df)

Now progress... 0
Now progress... 1000
Now progress... 2000

/mnt/data/emma/bilab/Steph_C/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/mnt/data/emma/bilab/Steph_C/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  



Now progress... 3000
Now progress... 4000
Now progress... 5000
Now progress... 6000
Now progress... 7000
Now progress... 8000
Now progress... 9000
Now progress... 10000
Now progress... 11000
Now progress... 0
Now progress... 1000
Now progress... 2000
Now progress... 3000
Now progress... 4000
Now progress... 5000
Now progress... 6000
Now progress... 7000


In [12]:
import torch.nn.functional as F

def get_cosine_similarity(df):
    df['cosine_sim'] = ''
    
    for idx , row in df.iterrows():
        if idx % 1000 ==0 :
            print(f'Now progress... {idx}')
        df['cosine_sim'][idx]= F.cosine_similarity(row.res_emb.unsqueeze(0), row.loc_emb.unsqueeze(0))
    return df

In [13]:
train_df = get_cosine_similarity(train_df)
test_df = get_cosine_similarity(test_df)

Now progress... 0


/mnt/data/emma/bilab/Steph_C/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


Now progress... 1000
Now progress... 2000
Now progress... 3000
Now progress... 4000
Now progress... 5000
Now progress... 6000
Now progress... 7000
Now progress... 8000
Now progress... 9000
Now progress... 10000
Now progress... 11000
Now progress... 0
Now progress... 1000
Now progress... 2000
Now progress... 3000
Now progress... 4000
Now progress... 5000
Now progress... 6000
Now progress... 7000


In [14]:
train_df.to_pickle('../data/Train_by_postoal_code_pointwise_v3_3.pkl')
test_df.to_pickle('../data/Test_by_postoal_code_pointwise_v3_3.pkl')